# ELYZA-tasks-100 による性能評価

In [ ]:
!pip install accelerate datasets jsonlines sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00


## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

> 📒 **ノート**
>
> この方法は、質問を直接入力する場合や、全モデルで共通のプロンプトを用いる場合に比べて公平性が劣る。
> しかし、実利用時は各モデルが最もよい性能を発揮するように、各モデルのプロンプトをチューニングすることが想定される。
> モデルカード記載のプロンプトをチューニング済みのプロンプトと見なすことで、実利用時の性能を比較することを狙いとする。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "llm-jp/llm-jp-13b-instruct-full-dolly-oasst-v1.0"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    device_map="auto")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
    # trust_remote_code=True, # ⚠️ Microsoft/Phi-1.5 のみ必要
)

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    prompt_template = "{input}### 回答："
    return prompt_template.format(input=input_text)

def generate_text(input_text):
    formatted_prompt = format_prompt(input_text)
    input_ids = tokenizer.encode(
        formatted_prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    tokens = model.generate(
        input_ids.to(device=model.device),
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][len(input_ids[0]):]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            print(generated_text)
            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.25M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/331 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/36.3k [00:00<?, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.86G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/5.87G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

1.目標を持つ
2.仕事の意味を考える
3.仕事を楽しむ
4.目標を達成するための計画を立てる
5.目標達成のために自分の限界に挑戦する
アザラシはクマと友達になる。クマはアザラシを家に連れて帰る。
古代ギリシャについて学ぶことは、その多様性と複雑さのために、興味深く、興味深く、挑戦的なことが多い。古代ギリシャ人が信じていたことや信じていたことは、現代の世界観や哲学とは異なるため、古代ギリシャを学ぶ際には、現代の知識を持ちながら、その世界観を理解し、受け入れることが重要である。古代ギリシャの知識は、古代ギリシャの哲学、文学、芸術の研究に役立ちます。古代ギリシャ語の文法や語彙を学ぶことで、古代ギリシャの文化や歴史に関する理解を深めることができます。また、古代ギリシャの哲学、文学、芸術の研究は、その哲学や文化の根底にある考え方や価値観を理解するのに役立ちます。
ゴミ圧縮機を購入する前に知っておくべきことは？
1.2001年宇宙の旅
2.ファウンデーション
3.アンドロイドは電気羊の夢を見るか？
4.華氏451
5.ソラリス
6.アンドロイドの夢の羊
7.惑星ソラリス
8.アンドロイドは電気羊の夢を見るか？
9.華氏451
10.銀河ヒッチハイク・ガイド
2
<a>だから
微積分学101, 美術史
2023年2月24日から26日
バンクーバー、ロンドン、名古屋
GPSトラッカー、子供、音声通話
HTMLはHypertext Markup Languageの略。
今日は, 教派, 京は
瞑想することだ。
盲人たちが互いに意見を戦わせるのは、彼らの目が見えないため、間違った意見を信じやすくなっているからである。
申し訳ない。
ある日、主人公の父親が突然亡くなり、主人公は父親の遺産を相続することになりました。しかし、父親の遺産は、父が生前に作った演劇の脚本でした。

主人公は、父親が作った演劇の脚本を読み始めると、父親の過去の出来事が明らかになります。そして父の過去が、主人公の人生に大きな影響を与えていきます。

主人公は、父親が作った演劇の脚本を読み進めていくうちに、物語の世界に引き込まれていきます。そして、自分の人生とその意味について深く考えさせられます。

主人公は、父親が作った演劇の脚本に描かれた世界に没頭し、自分の人生とその意味について深く考えるようになりました。そし